In [1]:
import pandas as pd
import numpy as np
from ollama import chat
from ollama import ChatResponse
import sys
import pprint
from tqdm import tqdm
from matplotlib import pyplot as plt
import json

sys.path.append("/Users/apurvgandhi/connections-solver/")
import simulator

In [2]:
system_prompt = """Solve today’s NYT Connections game. Here are the instructions for how to play this game:
Find groups of four items that share something in common.

Category Examples:
FISH: Bass, Flounder, Salmon, Trout
FIRE ___: Ant, Drill, Island, Opal

Categories will always be more specific than ‘5-LETTER-WORDS,’ ‘NAMES,’ or ‘VERBS.’

Example 1:
Words: [‘DART’, ‘HEM’, ‘PLEAT’, ‘SEAM’, ‘CAN’, ‘CURE’, ‘DRY’, ‘FREEZE’, ‘BITE’, ‘EDGE’, ‘PUNCH’, ‘SPICE’, ‘CONDO’, ‘HAW’, ‘HERO’, ‘LOO’]
Groupings:
Things to sew: [‘DART’, ‘HEM’, ‘PLEAT’, ‘SEAM’]
Ways to preserve food: [‘CAN’, ‘CURE’, ‘DRY’, ‘FREEZE’]
Sharp quality: [‘BITE’, ‘EDGE’, ‘PUNCH’, ‘SPICE’]
Birds minus last letter: [‘CONDO’, ‘HAW’, ‘HERO’, ‘LOO’]

Example 2:
Words: [‘COLLECTIVE’, ‘COMMON’, ‘JOINT’, ‘MUTUAL’, ‘CLEAR’, ‘DRAIN’, ‘EMPTY’, ‘FLUSH’, ‘CIGARETTE’, ‘PENCIL’, ‘TICKET’, ‘TOE’, ‘AMERICAN’, ‘FEVER’, ‘LUCID’, ‘PIPE’]
Groupings:
Shared: [‘COLLECTIVE’, ‘COMMON’, ‘JOINT’, ‘MUTUAL’]
Rid of contents: [‘CLEAR’, ‘DRAIN’, ‘EMPTY’, ‘FLUSH’]
Associated with “stub”: [‘CIGARETTE’, ‘PENCIL’, ‘TICKET’, ‘TOE’]
__ Dream: [‘AMERICAN’, ‘FEVER’, ‘LUCID’, ‘PIPE’]

Example 3:
Words: [‘HANGAR’, ‘RUNWAY’, ‘TARMAC’, ‘TERMINAL’, ‘ACTION’, ‘CLAIM’, ‘COMPLAINT’, ‘LAWSUIT’, ‘BEANBAG’, ‘CLUB’, ‘RING’, ‘TORCH’, ‘FOXGLOVE’, ‘GUMSHOE’, ‘TURNCOAT’, ‘WINDSOCK’]
Groupings:
Parts of an airport: [‘HANGAR’, ‘RUNWAY’, ‘TARMAC’, ‘TERMINAL’]
Legal terms: [‘ACTION’, ‘CLAIM’, ‘COMPLAINT’, ‘LAWSUIT’]
Things a juggler juggles: [‘BEANBAG’, ‘CLUB’, ‘RING’, ‘TORCH’]
Words ending in clothing: [‘FOXGLOVE’, ‘GUMSHOE’, ‘TURNCOAT’, ‘WINDSOCK’]
Categories share commonalities:

There are 4 categories of 4 words each.
Every word will be in only 1 category.
One word will never be in two categories.
There may be red herrings (words that seem to belong together but actually are in separate categories).
There may be compound words with a common prefix or suffix word.
A few other common categories include word and letter patterns, pop culture clues (such as music and movie titles), and fill-in-the-blank phrases.
You will be given a new example (Example 4) with today’s list of words. First, explain your reason for each category and then give your final answer following the structure below (Replace Category 1, 2, 3, 4 with their names instead):

Groupings:
Category1: [word1, word2, word3, word4]
Category2: [word5, word6, word7, word8]
Category3: [word9, word10, word11, word12]
Category4: [word13, word14, word15, word16]

Remember that the same word cannot be repeated across multiple categories, and you need to output 4 categories with 4 distinct words each. Also, do not make up words not in the list. This is the most important rule. Please obey.
All words and explanations must be in English
"""

user_prompt="""Example 4:
Words: {words}
Groupings"""

In [3]:
puzzles = simulator.load_puzzles("/Users/apurvgandhi/connections-solver/puzzles.csv")

In [6]:
puzzle_idx = 0
sample_words = puzzles[puzzle_idx].all_words
print(sample_words)

['SNOW', 'LEVEL', 'SHIFT', 'KAYAK', 'HEAT', 'TAB', 'BUCKS', 'RETURN', 'JAZZ', 'HAIL', 'OPTION', 'RAIN', 'SLEET', 'RACECAR', 'MOM', 'NETS']


In [5]:
for i in range(100):
    sample_words = puzzles[i].all_words
    if "WITCH" in sample_words:
        print(sample_words)
        print(i)
        break

['WOODSTOCK', 'GAMBIT', 'NEW YORK', 'GIANT', 'QUEEN', "ROCK 'N ROLL", 'WOLF', 'CHARLIE', 'YOU', 'PEPPERMINT PATTY', 'MATE', 'PRINCESS', 'PIGPEN', 'WITCH', 'LUCY', 'BISHOP']
96


In [7]:
input_user_prompt = user_prompt.format(words=sample_words)

In [8]:
from ollama import generate

In [14]:
stream = generate(
    model="qwq",
    prompt=input_user_prompt,
    system=system_prompt,
    stream=True,
    options={"temperature": 1, "num_predict": 1024, "min_p": 0.05, "repeat_last_n": 1024, "seed": 0},
    context=[],
    format="json"
)

In [15]:
res = ""
for chunk in stream:
    resp = chunk['response']
    res += resp
    print(resp, end='', flush=True)

{
    "categories": [
        {
            "name": "Palindromes",
            "words": ["KAYAK", "RACECAR", "MOM"]
        },
        {
            "name": "Weather Conditions",
            "words": ["SNOW", "HEAT", "HAIL", "RAIN", "SLEET"]
        },
        {
            "name": "Financial Terms",
            "words": ["LEVEL", "SHIFT", "TAB", "BUCKS", "RETURN"]
        },
        {
            "name": "Music Genres",
            "words": ["JAZZ"]
        }
    ],
    "explanation": "I've identified four main categories based on the given words. First, I noticed that some words are palindromes, meaning they read the same forwards and backwards. These include 'KAYAK', 'RACECAR', and 'MOM'. Second, several words relate to weather conditions, such as 'SNOW', 'HEAT', 'HAIL', 'RAIN', and 'SLEET'. Third, there are terms related to finance, including 'LEVEL', 'SHIFT', 'TAB', 'BUCKS', and 'RETURN'. Lastly, 'JAZZ' stands out as a music genre. However, since categories need to have four words

In [11]:
system_prompt2 = """
You are a helpful assistant who identifies 4 categories each containing 4 words from the input text.
There are 4 categories of 4 words each.
Every word will be in only 1 category.
One word will never be in two categories.

You will be given an input text with reasoning. First, explain your reason for each category and then give your final answer as JSON following the structure below (Replace Category 1, 2, 3, 4 with their names instead):

Groupings:
Category1: [word1, word2, word3, word4]
Category2: [word5, word6, word7, word8]
Category3: [word9, word10, word11, word12]
Category4: [word13, word14, word15, word16]

Remember that the same word cannot be repeated across multiple categories, and you need to output 4 categories with 4 distinct words each. Also, do not make up words not in the list. This is the most important rule. Please obey.
"""

input_prompt2 = """
{res}

Groupings:
"""

In [12]:
stream2 = generate(
    model="llama3.2",
    prompt=input_prompt2.format(res=res),
    system=system_prompt2,
    stream=False,
    options={"temperature": 0, "num_predict": 1024, "min_p": 0.05, "repeat_last_n": 1024, "seed": 0},
    context=[],
    format="json"
)

In [13]:
json.loads(stream2["response"]).values()

dict_values([{'Category1': ['SNOW', 'HAIL', 'RAIN', 'SLEET'], 'Category2': ['LEVEL', 'KAYAK', 'RACECAR', 'MOM'], 'Category3': ['SHIFT', 'RETURN', 'OPTION', 'TAB'], 'Category4': ['BUCKS', 'JAZZ', 'NETS', 'HEAT']}])

In [ ]:
json.loads(stream2["response"]).values()

In [ ]:
raw_response = json.loads(stream2["response"])
print(raw_response)
outputs = raw_response.values()
output_list = list(list(outputs)[0].values())

In [ ]:
output_list

In [ ]:
outputs = []
outputs.append(json.loads(stream2["response"])["categories"]